In [1]:
from splinter import Browser 
from bs4 import BeautifulSoup as bs
import time
import numpy as np
import pandas as pd
import collections
import pymongo
from IPython.display import display_html

In [2]:
# Visit URL
browser = Browser('chrome')

def init_browser():
    # @NOTE: Replace the path with your actual path to the chromedriver
    executable_path = {"executable_path": "chromedriver"}
    return Browser("chrome", **executable_path)

In [3]:
def scrape_nasa():
    browser = init_browser()
    url = "https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest"
    browser.visit(url)
    
    time.sleep(1)
    
    # Scrape page into Soup
    html = browser.html
    soup = bs(html, "html.parser")
    
    #get the news title
    news_title = str(soup.find('div',class_="content_title").text)
    #get the corrsponding paragraph text
    news_p = str(soup.find('div',class_="article_teaser_body").text)
    
       # Close the browser after scraping
    browser.quit()

    # Return results
    return news_title, news_p

("Six Things to Know About NASA's Opportunity Rover",
 "Opportunity's mission is complete. Here are highlights from its time on Mars.")

In [4]:
def scrape_JPL():
    browser = init_browser()
    url = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
    browser.visit(url)
    
    time.sleep(1)
    
    # Scrape page into Soup
    html = browser.html
    soup = bs(html, "html.parser")
    anchor = soup.find('a', class_="button fancybox")
    image_url = str(anchor.get('data-fancybox-href'))
    featured_image_url = ("https://www.jpl.nasa.gov/" + image_url)
    browser.quit()
    
    return featured_image_url

'https://www.jpl.nasa.gov//spaceimages/images/mediumsize/PIA16613_ip.jpg'

In [5]:
def scrape_twitter():
    browser = init_browser()
    url = "https://twitter.com/marswxreport?lang=en"
    browser.visit(url)
    
    time.sleep(1)
    
    # Scrape page into Soup
    html = browser.html
    soup = bs(html, "html.parser")
    mars_weather = soup.find("p", class_="TweetTextSize TweetTextSize--normal js-tweet-text tweet-text").text
    mars_weather = mars_weather.strip('/n')
    browser.quit()
    return mars_weather

u'#Opportunity the robot achieved so much because of #Opportunity the team.  People like @PlanetaryKeri, @doug_ellison, @marsroverdriver, @tanyaofmars,  @marssciencegrad, @cirquelar, @bellutta, @Dr_ThomasZ and so many more. Read more at #WRALhttps://www.wral.com/my-battery-is-low-and-it-s-getting-dark-end-of-opportunity-mission-a-loss-for-team/18197530/\xa0\u2026'

In [12]:
def scrape_table():
    browser = init_browser()
    url = "https://space-facts.com/mars/"
    browser.visit(url)
    
    time.sleep(1)
    
    # Scrape page into Soup
    html = browser.html
    soup = bs(html, "html.parser")
    table = soup.find_all('table')[0]
    df = pd.read_html(str(table),skiprows=1)[0]
    df = df.rename(index=str, columns={0: "Description", 1: "Data"})
    html_table = df.to_html(header=True,index=False,index_names=False,justify="center")
    html_table = html_table.replace('\n'," ").encode('utf-8')
    html_table = str(html_table)
    browser.quit()
    return html_table

scrape_table()

'<table border="1" class="dataframe">   <thead>     <tr style="text-align: center;">       <th>Description</th>       <th>Data</th>     </tr>   </thead>   <tbody>     <tr>       <td>Polar Diameter:</td>       <td>6,752 km</td>     </tr>     <tr>       <td>Mass:</td>       <td>6.42 x 10^23 kg (10.7% Earth)</td>     </tr>     <tr>       <td>Moons:</td>       <td>2 (Phobos &amp; Deimos)</td>     </tr>     <tr>       <td>Orbit Distance:</td>       <td>227,943,824 km (1.52 AU)</td>     </tr>     <tr>       <td>Orbit Period:</td>       <td>687 days (1.9 years)</td>     </tr>     <tr>       <td>Surface Temperature:</td>       <td>-153 to 20 \xc3\x82\xc2\xb0C</td>     </tr>     <tr>       <td>First Record:</td>       <td>2nd millennium BC</td>     </tr>     <tr>       <td>Recorded By:</td>       <td>Egyptian astronomers</td>     </tr>   </tbody> </table>'

In [27]:
def image_urls():
    link_names = ['Cerberus Hemisphere Enhanced','Schiaparelli Hemisphere Enhanced','Syrtis Major Hemisphere Enhanced','Valles Marineris Hemisphere Enhanced']
    url_list = []
    title_list = []
    browser = init_browser()
    url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"
    browser.visit(url)
    for name in link_names:
        browser.click_link_by_partial_text(name)
        time.sleep(1)
        # Scrape page into Soup for img url
        html = browser.html
        soup = bs(html, "html.parser")
        img = soup.find('img',class_="wide-image")
        src = str(img.get('src'))
        img_url = ("https://astrogeology.usgs.gov/" + src)
        url_list.append(img_url)
        
        title = soup.find('h2',class_="title").text
        title_list.append(str(title))
        
        browser.back()
    browser.quit()
    hemisphere_image_urls = [
    {"title": title_list[0], "img_url": url_list[0]},
    {"title": title_list[1], "img_url": url_list[1]},
    {"title": title_list[2], "img_url": url_list[2]},
    {"title": title_list[3], "img_url": url_list[3]},
    ]
    
    return hemisphere_image_urls

[{'img_url': 'https://astrogeology.usgs.gov//cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg',
  'title': 'Cerberus Hemisphere Enhanced'},
 {'img_url': 'https://astrogeology.usgs.gov//cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg',
  'title': 'Schiaparelli Hemisphere Enhanced'},
 {'img_url': 'https://astrogeology.usgs.gov//cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg',
  'title': 'Syrtis Major Hemisphere Enhanced'},
 {'img_url': 'https://astrogeology.usgs.gov//cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg',
  'title': 'Valles Marineris Hemisphere Enhanced'}]

In [29]:
def scrape():
    dictionary = {"title and teaser":scrape_nasa(),
                  "featured image url":scrape_JPL(),
                  "mars weather":scrape_twitter(),
                  "image_urls":image_urls()}
    return dictionary

In [30]:
scrape()

{'featured image url': 'https://www.jpl.nasa.gov//spaceimages/images/mediumsize/PIA19101_ip.jpg',
 'image_urls': [{'img_url': 'https://astrogeology.usgs.gov//cache/images/cfa62af2557222a02478f1fcd781d445_cerberus_enhanced.tif_full.jpg',
   'title': 'Cerberus Hemisphere Enhanced'},
  {'img_url': 'https://astrogeology.usgs.gov//cache/images/3cdd1cbf5e0813bba925c9030d13b62e_schiaparelli_enhanced.tif_full.jpg',
   'title': 'Schiaparelli Hemisphere Enhanced'},
  {'img_url': 'https://astrogeology.usgs.gov//cache/images/ae209b4e408bb6c3e67b6af38168cf28_syrtis_major_enhanced.tif_full.jpg',
   'title': 'Syrtis Major Hemisphere Enhanced'},
  {'img_url': 'https://astrogeology.usgs.gov//cache/images/7cf2da4bf549ed01c17f206327be4db7_valles_marineris_enhanced.tif_full.jpg',
   'title': 'Valles Marineris Hemisphere Enhanced'}],
 'mars weather': u'#Opportunity the robot achieved so much because of #Opportunity the team.  People like @PlanetaryKeri, @doug_ellison, @marsroverdriver, @tanyaofmars,  @mars